In [3]:
import numpy as np
import pandas as pd
import seaborn as sns

import petab
import pypesto
import pypesto.petab


In [4]:
# rng seed
np.random.seed(500)

Import petab files and define the problem

In [ ]:
petab_yaml = 'petab_files/baseline_model.yaml'
petab.validate(petab_yaml)
petab_problem = petab.Problem.from_yaml(petab_yaml)

problem = pypesto.petab.PetabImporter(
        petab_problem,
        hierarchical=False,
        model_name=f"Baseline_Model",
    ).create_problem(force_compile=True, verbose=False)
problem.objective.amici_model.setAllStatesNonNegative()

Compiling amici model to folder /Users/yuhongliu/Documents/OV/model/baseline_model/amici_models/0.27.0/Baseline_Model.
2024-11-20 16:22:33.194 - amici.petab.sbml_import - INFO - Importing model ...
2024-11-20 16:22:33.195 - amici.petab.sbml_import - INFO - Validating PEtab problem ...
2024-11-20 16:22:33.206 - amici.petab.sbml_import - INFO - Model name is 'Baseline_Model'.
Writing model code to '/Users/yuhongliu/Documents/OV/model/baseline_model/amici_models/0.27.0/Baseline_Model'.
2024-11-20 16:22:33.207 - amici.petab.sbml_import - INFO - Species: 0
2024-11-20 16:22:33.207 - amici.petab.sbml_import - INFO - Global parameters: 10
2024-11-20 16:22:33.207 - amici.petab.sbml_import - INFO - Reactions: 0
2024-11-20 16:22:33.222 - amici.petab.sbml_import - INFO - Observables: 1
2024-11-20 16:22:33.223 - amici.petab.sbml_import - INFO - Sigmas: 1
2024-11-20 16:22:33.223 - amici.petab.sbml_import - DEBUG - Adding output parameters to model: ['sigma_a', 'sigma_b']
2024-11-20 16:22:33.224 - am

running build_ext
running AmiciBuildCMakeExtension
------------------------------ model_ext ------------------------------

==> Configuring:
$ cmake -S /Users/yuhongliu/Documents/OV/model/baseline_model/amici_models/0.27.0/Baseline_Model -B /Users/yuhongliu/Documents/OV/model/baseline_model/amici_models/0.27.0/Baseline_Model/build_model_ext -G Ninja -DCMAKE_MAKE_PROGRAM=/Users/yuhongliu/Documents/venv_ov/bin/ninja -DCMAKE_BUILD_TYPE=Release -DCMAKE_INSTALL_PREFIX:PATH=/Users/yuhongliu/Documents/OV/model/baseline_model/amici_models/0.27.0/Baseline_Model/Baseline_Model -DCMAKE_VERBOSE_MAKEFILE=ON -DCMAKE_PREFIX_PATH=/Users/yuhongliu/Documents/venv_ov/lib/python3.12/site-packages/amici -DAMICI_PYTHON_BUILD_EXT_ONLY=ON -DPython3_EXECUTABLE=/Users/yuhongliu/Documents/venv_ov/bin/python3

==> Building:
$ cmake --build /Users/yuhongliu/Documents/OV/model/baseline_model/amici_models/0.27.0/Baseline_Model/build_model_ext --config Release

==> Installing:
$ cmake --install /Users/yuhongliu/Docum

2024-11-20 16:22:45.974 - amici.petab.sbml_import - INFO - Finished Importing PEtab model            (1.28E+01s)


# Gradient Check

In [10]:
# obtain random startpoints
startpoints = problem.get_startpoints(n_starts=2)

In [11]:
problem.objective.check_grad(
    x = startpoints[0],
    eps = 1e-5,  # default
    verbosity = 0,
)

,grad,fd_f,fd_b,fd_c,fd_err,abs_err,rel_err
rho,-15372.873517,-15533.773102,-15212.110440,-15372.941771,321.662661,0.068254,4.439863e-06
kappa,-68.346074,-230.424625,91.773285,-69.325670,322.197910,0.979596,1.413035e-02
psi,9.708414,-151.685904,170.334218,9.324157,322.020122,0.384257,4.121088e-02
beta,0.000014,-160.675306,161.252567,0.288630,321.927873,0.288616,9.999169e-01
alpha,48390.305366,48227.563333,48547.531452,48387.547392,319.968120,2.757973,5.699759e-05
delta,-0.008159,-142.537583,154.712104,6.087261,297.249687,6.095420,1.001339e+00
sigma_a,-36156.022078,-36316.323156,-35995.716511,-36156.019833,320.606645,0.002245,6.208419e-08
sigma_b,-6073.862915,-6234.850518,-5912.864856,-6073.857687,321.985662,0.005228,8.607549e-07


In [12]:
problem.objective.check_grad(
    x = problem.get_reduced_vector(startpoints[0]),
    eps = 1e-5,  # default
    verbosity = 0,
)

,grad,fd_f,fd_b,fd_c,fd_err,abs_err,rel_err
rho,-15372.873517,-15533.773102,-15212.110440,-15372.941771,321.662661,0.068254,4.439863e-06
kappa,-68.346074,-230.424625,91.773285,-69.325670,322.197910,0.979596,1.413035e-02
psi,9.708414,-151.685904,170.334218,9.324157,322.020122,0.384257,4.121088e-02
beta,0.000014,-160.675306,161.252567,0.288630,321.927873,0.288616,9.999169e-01
alpha,48390.305366,48227.563333,48547.531452,48387.547392,319.968120,2.757973,5.699759e-05
delta,-0.008159,-142.537583,154.712104,6.087261,297.249687,6.095420,1.001339e+00
sigma_a,-36156.022078,-36316.323156,-35995.716511,-36156.019833,320.606645,0.002245,6.208419e-08
sigma_b,-6073.862915,-6234.850518,-5912.864856,-6073.857687,321.985662,0.005228,8.607549e-07


In [13]:
gc = problem.objective.check_grad_multi_eps(
    x=startpoints[0],
    verbosity=0,
    label='rel_err',  # default
)
gc

,grad,fd_f,fd_b,fd_c,fd_err,abs_err,rel_err,eps
rho,-15372.873517,-1.553377e+04,-1.521211e+04,-15372.941771,3.216627e+02,0.068254,4.439863e-06,1.000000e-05
kappa,-68.346074,-6.979364e+01,-6.688834e+01,-68.340988,2.905298e+00,0.005086,7.442590e-05,1.000000e-03
psi,9.708414,7.952549e+00,1.133660e+01,9.644572,3.384047e+00,0.063841,6.618733e-03,1.000000e-03
beta,0.000014,-1.075858e-02,2.182030e-02,0.005531,3.257888e-02,0.005517,5.227750e-02,1.000000e-01
alpha,48390.305366,4.822756e+04,4.854753e+04,48387.547392,3.199681e+02,2.757973,5.699759e-05,1.000000e-05
delta,-0.008159,-7.884762e-03,-6.415637e-03,-0.007150,1.469125e-03,0.001009,1.086641e-02,1.000000e-01
sigma_a,-36156.022078,-1.646835e+06,1.574523e+06,-36156.022361,3.221358e+06,0.000283,7.820785e-09,1.000000e-09
sigma_b,-6073.862915,-1.616753e+06,1.604605e+06,-6073.859367,3.221358e+06,0.003548,5.840816e-07,1.000000e-09


In [14]:
def highlight_value_above_threshold(x, threshold=10):
    return ['color: darkorange' if xi > threshold else None for xi in x]

gc.style.apply(
    highlight_value_above_threshold, subset=["fd_err"],
).background_gradient(
    cmap=sns.light_palette("purple", as_cmap=True), subset=["abs_err"],
).background_gradient(
    cmap=sns.light_palette("red", as_cmap=True), subset=["rel_err"],
)

,grad,fd_f,fd_b,fd_c,fd_err,abs_err,rel_err,eps
rho,-15372.873517,-15533.773102,-15212.110440,-15372.941771,321.662661,0.068254,0.000004,0.000010
kappa,-68.346074,-69.793637,-66.888339,-68.340988,2.905298,0.005086,0.000074,0.001000
psi,9.708414,7.952549,11.336596,9.644572,3.384047,0.063841,0.006619,0.001000
beta,0.000014,-0.010759,0.021820,0.005531,0.032579,0.005517,0.052278,0.100000
alpha,48390.305366,48227.563333,48547.531452,48387.547392,319.968120,2.757973,0.000057,0.000010
delta,-0.008159,-0.007885,-0.006416,-0.007150,0.001469,0.001009,0.010866,0.100000
sigma_a,-36156.022078,-1646835.158681,1574523.113959,-36156.022361,3221358.272640,0.000283,0.000000,0.000000
sigma_b,-6073.862915,-1616752.993868,1604605.275134,-6073.859367,3221358.269002,0.003548,0.000001,0.000000
